Download to PDF

In [ ]:
from reportlab.pdfgen import canvas

def create_pdf(letter):
    c = canvas.Canvas(f"{letter}.pdf")
    c.drawString(100, 750, letter)
    c.save()

letters = ['A', 'B', 'C', 'D', 'E', 'F']
for letter in letters:
    create_pdf(letter)

Download to PNG

In [ ]:
from PIL import Image, ImageDraw, ImageFont

def create_image(letter):
    img = Image.new('RGB', (200, 200), color=(255, 255, 255))
    d = ImageDraw.Draw(img)
    try:
        font = ImageFont.truetype("arial.ttf", 100)
    except IOError:
        font = ImageFont.load_default()
    bbox = d.textbbox((0, 0), letter, font=font)
    text_width, text_height = bbox[2] - bbox[0], bbox[3] - bbox[1]
    position = ((img.width - text_width) // 2, (img.height - text_height) // 2)
    d.text(position, letter, fill=(0, 0, 0), font=font)
    img.save(f"{letter}.png")

letters = ['A', 'B', 'C', 'D', 'E', 'F']
for letter in letters:
    create_image(letter)


Downloading Stock Pictures

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt

def create_stock_chart(ticker):
    stock_data = yf.download(ticker, period='6mo')
    
    plt.figure(figsize=(10, 6))
    plt.plot(stock_data['Close'], label=ticker)
    plt.title(f'{ticker} Stock Price - Last 6 Months')
    plt.xlabel('Date')
    plt.ylabel('Close Price (USD)')
    plt.legend()
    plt.grid()
    
    plt.savefig(f"{ticker}.png")
    plt.close()

tech_stocks = ['AAPL', 'MSFT', 'NVDA']

for ticker in tech_stocks:
    create_stock_chart(ticker)


Download better pictures

In [ ]:
import yfinance as yf
import pandas as pd
import mplfinance as mpf
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import io

def get_stock_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="max")
        market_cap = stock.info.get('marketCap', 'N/A')
        sector = stock.info.get('sector', 'N/A')
        eps = stock.info.get('trailingEps', 'N/A')
        revenue = stock.info.get('totalRevenue', 'N/A')

    except Exception as e:
        print(f"Error retrieving data for {ticker}: {e}")
        return None

    hist['MarketCap'] = market_cap

    def get_highs(hist, period):
        return hist['High'].rolling(window=period).max().dropna()

    high_1m = get_highs(hist, 21)
    high_5m = get_highs(hist, 105)
    high_60m = get_highs(hist, 1260)

    return {
        "Ticker": ticker,
        "Sector": sector,
        "EPS": eps,
        "Revenue": revenue,
        "MarketCap": market_cap,
        "HistoricalData": hist,
        "High1M": high_1m,
        "High5M": high_5m,
        "High60M": high_60m
    }

def create_stock_image(stock_data):
    fig, ax = plt.subplots(figsize=(12, 8))

    mpf.plot(stock_data['HistoricalData'], type='candle', ax=ax, volume=False, show_nontrading=False)

    text = (
        f"Ticker: {stock_data['Ticker']}\n"
        f"Sector: {stock_data['Sector']}\n"
        f"EPS: {stock_data['EPS']}\n"
        f"Revenue: {stock_data['Revenue']}\n"
        f"MarketCap: {stock_data['MarketCap']}\n"
    )
    ax.annotate(text, xy=(0.02, 0.98), xycoords='axes fraction', fontsize=10, va='top', ha='left', bbox=dict(facecolor='white', alpha=0.8))

    filename = f"{stock_data['Ticker']}.png"
    plt.savefig(filename)
    plt.close()

tech_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA']

for ticker in tech_stocks:
    data = get_stock_data(ticker)
    if data:
        create_stock_image(data)
    else:
        print(f"Failed to retrieve data for {ticker}")

Old way of getting data

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mdates
from datetime import datetime, timedelta

exchange_mapping = {
    'NMS': 'NASDAQ',
    'NYQ': 'NYSE',
}

sector_indices = pd.read_csv('sectors.csv', index_col='sector')

def fetch_stock_data(ticker_symbol, start_date, length):
    try:
        start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')
        end_date_obj = start_date_obj + timedelta(days=length)
        end_date = end_date_obj.strftime('%Y-%m-%d')
        
        ticker = yf.Ticker(ticker_symbol)
        daily_data = ticker.history(start=start_date, end=end_date)
        
        financials = ticker.quarterly_financials.T.sort_index(ascending=False).head(10)
        
        if financials.empty:
            raise ValueError(f"No financial data available for {ticker_symbol}")
        
        float_shares = ticker.info.get('floatShares', 'N/A')
        sector = ticker.info.get('sector', 'N/A')
        exchange_code = ticker.info.get('exchange', 'N/A')
        exchange = exchange_mapping.get(exchange_code, exchange_code)
        
        earnings_dates = ticker.earnings_dates.sort_index(ascending=False).head(4) if not ticker.earnings_dates.empty else None
        sector_index_ticker = sector_indices.loc[sector, 'index'] if sector in sector_indices.index else None
        sector_data = yf.Ticker(sector_index_ticker).history(start=start_date, end=end_date) if sector_index_ticker else None
        
        return daily_data, financials, float_shares, sector_data, sector, exchange, earnings_dates
    except Exception as e:
        print(f"Error fetching data for {ticker_symbol}: {e}")
        return None, None, None, None, None, None, None

def plot_stock_data(ticker_symbol, daily_data, financial_data, float_shares, sector_data, sector, exchange, earnings_dates):
    plt.style.use('dark_background')
    fig, axes = plt.subplots(6, 1, figsize=(24, 36), gridspec_kw={'height_ratios': [2.5, 0.6, 0.6, 0.6, 0.6, 0.6]})
    ax1, ax2, ax3, ax4, ax5, ax6 = axes

    def format_date_axis(ax):
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        ax.set_facecolor('black')
    
    if not daily_data.empty:
        daily_data['Date'] = mdates.date2num(daily_data.index.to_pydatetime())
        candlestick_ohlc(ax1, daily_data[['Date', 'Open', 'High', 'Low', 'Close']].values, width=0.6, colorup='green', colordown='red')
        ax1.set_title(f'{ticker_symbol}\nSector: {sector}\nExchange: {exchange}', fontsize=30)
        ax1.set_ylabel('Price', fontsize=20)

        if sector_data is not None:
            sector_data['Date'] = mdates.date2num(sector_data.index.to_pydatetime())
            ax6.plot(sector_data['Date'], sector_data['Close'], color='white', linewidth=2)
            ax6.set_ylabel('Sector Price', fontsize=20)
        else:
            ax6.text(0.5, 0.5, f"No sector data available for {sector}", transform=ax6.transAxes, fontsize=20, ha='center', va='center', color='red')

        ax2.bar(daily_data['Date'], daily_data['Volume'], color='white', width=0.6)
        ax2.set_ylabel('Volume', fontsize=20)

        ax3.plot(daily_data['Date'], [float_shares] * len(daily_data), color='white', linewidth=2)
        ax3.set_ylabel('Float Shares', fontsize=20)

        financial_data['Basic EPS'] = financial_data['Basic EPS'].ffill().infer_objects(copy=False)
        financial_data['Total Revenue'] = financial_data['Total Revenue'].ffill().infer_objects(copy=False)

        if 'Basic EPS' in financial_data.columns:
            ax4.step(financial_data.index, financial_data['Basic EPS'], where='mid', color='white')
            ax4.set_ylabel('EPS', fontsize=20)
        else:
            ax4.text(0.5, 0.5, "EPS data not available", transform=ax4.transAxes, fontsize=20, ha='center', va='center', color='red')

        if 'Total Revenue' in financial_data.columns:
            ax5.step(financial_data.index, financial_data['Total Revenue']/1e9, where='mid', color='white')
            ax5.set_ylabel('Revenue (B)', fontsize=20)
        else:
            ax5.text(0.5, 0.5, "Revenue data not available", transform=ax5.transAxes, fontsize=20, ha='center', va='center', color='red')

        for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
            format_date_axis(ax)
    else:
        for ax in axes:
            ax.text(0.5, 0.5, f"Failed to fetch data for {ticker_symbol}", transform=ax.transAxes, fontsize=20, ha='center', va='center', color='red')

    plt.tight_layout(pad=4.0)
    plt.subplots_adjust(top=0.95)
    plt.savefig(f'{ticker_symbol}_data.png', dpi=300, bbox_inches='tight')
    plt.close()

def get_stock_data(ticker_symbol, start_date, length):
    data = fetch_stock_data(ticker_symbol, start_date, length)
    if data[0] is not None:
        plot_stock_data(ticker_symbol, *data)
    else:
        print(f"Failed to fetch data for {ticker_symbol}")

ticker_symbol = 'MSFT'
start_date = '2005-01-01'
length = 150
get_stock_data(ticker_symbol, start_date, length)
